In [1]:
#Roda ele
## BIBLIOTECAS
import requests
from bs4 import BeautifulSoup

#Manipulação de Dados
import pandas as pd
import numpy as np

import shutil
import os
import pyautogui

import unidecode #arrumar textos

#tempo
import time
from time import sleep

import re #regex

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import matplotlib.font_manager as fm #para alterar as fontes dos graficos
import seaborn as sns

import pyperclip

import statistics
import math

from scipy import stats

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


# Definindo o estilo de fonte e tema
fonte = fm.FontProperties(family="Calibri", style="italic")

sns.set_style('whitegrid') #cor da grade(fundo)

import pyperclip
from tableone import TableOne

#TESTE ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

import seaborn

from statsmodels.stats.stattools import durbin_watson #teste de autocorrelação

In [2]:
inicio = time.time() #iniciando o cronometro

df = pd.read_excel("BANCO CULTURA.xlsx")
df.sample(5)

fim = time.time() #encerrando cronometro
print('Tempo para rodar a simulação:', round(fim - inicio, 3))

Tempo para rodar a simulação: 109.012


In [3]:
len(df)

259269

In [4]:
print(df.columns)
display(df.sample(5))

Index(['Ano', 'Unidade',
       'Qual é sua principal área ou unidade neste hospital?',
       'Outro (especifique)',
       '1. As pessoas se apoiam umas às outras nesta unidade',
       '2. Temos pessoas suficientes para lidar com o volume de trabalho',
       '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
       '4. Nesta unidade, as pessoas se tratam com respeito',
       '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
       '6. Estamos ativamente buscando melhorias para a segurança do paciente',
       '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
       '8. Os funcionários sentem que seus erros são usados contra eles',
       '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
       '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
       '11. Quando uma área nesta unidade fi

,Ano,Unidade,Qual é sua principal área ou unidade neste hospital?,Outro (especifique),1. As pessoas se apoiam umas às outras nesta unidade,2. Temos pessoas suficientes para lidar com o volume de trabalho,"3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa","4. Nesta unidade, as pessoas se tratam com respeito",5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes,6. Estamos ativamente buscando melhorias para a segurança do paciente,...,SEÇÃO H: 4. Qual é o seu cargo neste hospital? Marque UMA resposta que melhor descreva a sua função.,Outro (especifique)2,"5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?","5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes? 3",6. Há quanto tempo você trabalha em sua profissão ou especialização atual?,"SEÇÃO I: Seus Comentários Por favor, sinta-se à vontade para fazer quaisquer comentários sobre a segurança do paciente, erros ou notificação de eventos em seu hospital.","5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes? 32",Regional,Estado,Acreditação
60820,2022,Santa Marina,Administrativa,Auxiliar administrativo de enfermagem,Concordo + Concordo totalmente,Nem um nem outro,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Discordo + Discordo totalmente,Concordo + Concordo totalmente,...,Auxiliar de Enfermagem,NaN,"a. SIM, tipicamente tenho contato ou interação...",NaN,b. 1 a 5 anos,Nada a declarar,"a. SIM, tipicamente tenho contato ou interação...",CO,MS,Não possui
123421,2020,Copa D'Or,Administrativa,NaN,Discordo + Discordo totalmente,Concordo + Concordo totalmente,Discordo + Discordo totalmente,Concordo + Concordo totalmente,Discordo + Discordo totalmente,Concordo + Concordo totalmente,...,Enfermeiro(a) Assistencial,NaN,"a. SIM, tipicamente tenho contato ou interação...",NaN,b. 1 a 5 anos,NaN,"a. SIM, tipicamente tenho contato ou interação...",SE,RJ,JCI
226613,2020,Vivalle,Unidade de Internação neonatal/ berçário,NaN,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Discordo + Discordo totalmente,Concordo + Concordo totalmente,...,Médico assistente - clínico,NaN,"a. SIM, tipicamente tenho contato ou interação...",NaN,b. 1 a 5 anos,Nenhuma,"a. SIM, tipicamente tenho contato ou interação...",SE,SP,Qmentum
335,2014,Santa Luzia,Centro Cirúrgico,NaN,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Concordo + Concordo totalmente,...,Enfermeiro(a) Assistencial,NaN,"a. SIM, tipicamente tenho contato ou interação...",NaN,a. Menos de 1 ano,Nada a declarar;,"a. SIM, tipicamente tenho contato ou interação...",CO,DF,ONA 2
144261,2021,Quinta D'Or,Unidade de terapia intensiva pediátrica,NaN,Discordo + Discordo totalmente,Concordo + Concordo totalmente,Concordo + Concordo totalmente,Discordo + Discordo totalmente,Concordo + Concordo totalmente,Concordo + Concordo totalmente,...,Técnico de Enfermagem,NaN,"a. SIM, tipicamente tenho contato ou interação...",NaN,d. 11 a 15 anos,Ter atenção redobrada no caso do recebimento d...,"a. SIM, tipicamente tenho contato ou interação...",SE,RJ,Qmentum


In [5]:
df = df.drop('Outro (especifique)', axis=1)
df = df.drop('Outro (especifique)2', axis=1)
df = df.drop('SEÇÃO I: Seus Comentários  Por favor, sinta-se à vontade para fazer quaisquer comentários sobre a segurança do paciente, erros ou  notificação de eventos em seu hospital.  ', axis=1)

df = df.drop(df[df['22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente'] == 'Algumas vezes'].index)
df = df.reset_index(drop=True)

df['5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?  '].fillna('b. NÃO, tipicamente não tenho contato ou interação direta com os pacientes.', inplace = True)
df = df.drop('5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?  3', axis=1)
df = df.drop('5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?  32', axis=1)

coluna = '5. Em sua função, você tipicamente tem contato ou interação direta com os pacientes?  '
counts = df[coluna].value_counts()
percentages = df[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
res = pd.DataFrame({'counts': counts, 'percentage': percentages})
print(res)

df_backup = df.copy()

#df = df.drop('', axis=1)

                                                    counts percentage
a. SIM, tipicamente tenho contato ou interação ...  199671      77.0%
b. NÃO, tipicamente não tenho contato ou intera...   59597      23.0%


In [6]:
coluna = ' SEÇÃO H: 4. Qual é o seu cargo neste hospital? Marque UMA resposta que melhor descreva a sua função.'
print(df[coluna].value_counts(),'\n')

df["Cargo"] = np.nan
df["Cargo"][df[coluna] == 'Enfermeiro Supervisor/Coordenador/Gerente'] = "Enfermeiro"
df["Cargo"][df[coluna] == 'Enfermeiro(a) Assistencial'] = "Enfermeiro"
df["Cargo"][df[coluna] == 'Técnico de Enfermagem'] = "Tecnico de Enfermeiro"
df["Cargo"][df[coluna] == 'Farmacêutico'] = "Outros"
df["Cargo"][df[coluna] == 'Ouvidor'] = "Outros"
df["Cargo"][df[coluna] == 'Assistente/Atendente/Secretária da Unidade'] = "Outros"
df["Cargo"][df[coluna] == 'Copeiro'] = "Outros"
df["Cargo"][df[coluna] == 'Administrativo (Recepções)'] = "Outros"
df["Cargo"][df[coluna] == 'Psicologo'] = "Outros"
df["Cargo"][df[coluna] == 'Médico plantonista terapia intensiva'] = "Médico"
df["Cargo"][df[coluna] == 'Radiologia)'] = "Outros"
df["Cargo"][df[coluna] == 'Fisioterapeuta'] = "Outros"
df["Cargo"][df[coluna] == 'Nutricionista'] = "Outros"
df["Cargo"][df[coluna] == 'Técnico de exames (ex.: Eletrocardiograma Laboratório'] = "Outros"
df["Cargo"][df[coluna] == 'Auxiliar de Enfermagem'] = "Tecnico Enfermeiro"
df["Cargo"][df[coluna] == 'Médico staff cirúrgico'] = "Médico"
df["Cargo"][df[coluna] == 'Médico Gerente'] = "Médico"
df["Cargo"][df[coluna] == 'Médico staff clínico'] = "Médico"
df["Cargo"][df[coluna] == 'Concierge'] = "Outros"
df["Cargo"][df[coluna] == 'Farmacêutico Supervisor/Coordenador/Gerente'] = "Outros"
df["Cargo"][df[coluna] == 'Médico assistente -  clínico'] = "Médico"
df["Cargo"][df[coluna] == 'Assistente social'] = "Outros"
df["Cargo"][df[coluna] == 'Nutricionista Supervisor/Coordenador/Gerente'] = "Outros"
df["Cargo"][df[coluna] == 'Médico plantonista emergência'] = "Médico"
df["Cargo"][df[coluna] == 'Médico assistente - cirúrgico'] = "Médico"
df["Cargo"][df[coluna] == 'Médico SADT'] = "Médico"
df["Cargo"][df[coluna] == 'Fonoaudiólogo'] = "Outros"
df["Cargo"][df[coluna] == 'Terapeuta Ocupacional'] = "Outros"

df = df.drop(' SEÇÃO H: 4. Qual é o seu cargo neste hospital? Marque UMA resposta que melhor descreva a sua função.', axis=1)

print(df['Cargo'].value_counts(), '\n')

Técnico de Enfermagem                                      80732
Enfermeiro(a) Assistencial                                 27869
Administrativo (Recepções)                                 15589
Médico plantonista emergência                               8072
Fisioterapeuta                                              7697
Assistente/Atendente/Secretária da Unidade                  7537
Médico plantonista terapia intensiva                        6011
Radiologia)                                                 5752
Auxiliar de Enfermagem                                      5425
Enfermeiro Supervisor/Coordenador/Gerente                   4816
Copeiro                                                     4673
Médico staff clínico                                        4220
Técnico de exames (ex.: Eletrocardiograma, Laboratório,     4205
Farmacêutico                                                3106
Médico staff cirúrgico                                      2797
Nutricionista            

In [7]:
sum(pd.isnull(df['Acreditação']))

0

In [8]:
df['Acreditação'][df['Acreditação'] == 'Não possui'] = 0
df['Acreditação'][df['Acreditação'] == 0] = 0

df['Acreditação2'] = df['Acreditação']

df['Acreditação'][df['Acreditação'] == 'ONA 2'] = 'ONA'
df['Acreditação'][df['Acreditação'] == 'ONA 3'] = 'ONA'
df['Acreditação'][df['Acreditação'] == 'ONA 1'] = 'ONA'
df['Acreditação'][df['Acreditação'] == 'Qmentum'] = 'Qmentum'
df['Acreditação'][df['Acreditação'] == 'JCI'] = 'JCI'

df['Acreditação2'][df['Acreditação2'] == 'N1'] = 'ONA 1'
df['Acreditação'][df['Acreditação'] == 'N1'] = 'ONA'

print(df['Acreditação'].value_counts(),'\n')
print(df['Acreditação2'].value_counts(),'\n')

###################################################3#
df['acreditacao'] = 0
df['acreditacao'][df['Acreditação'] != 0] = 1
print(df['acreditacao'].value_counts())

ONA        109283
Qmentum     67447
JCI         52947
0           29591
Name: Acreditação, dtype: int64 

ONA 3      75091
Qmentum    67447
JCI        52947
0          29591
ONA 2      25382
ONA 1       8810
Name: Acreditação2, dtype: int64 

1    229677
0     29591
Name: acreditacao, dtype: int64


In [9]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Ano,259268.0,NaN,NaN,NaN,2019.111267,2.299624,2014.0,2017.0,2019.0,2021.0,2022.0
Unidade,259268,71,Itaim,13456,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Qual é sua principal área ou unidade neste hospital?,224891,29,Unidade de terapia intensiva adulto,34413,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1. As pessoas se apoiam umas às outras nesta unidade,256790,3,Concordo + Concordo totalmente,177728,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2. Temos pessoas suficientes para lidar com o volume de trabalho,256996,3,Discordo + Discordo totalmente,123128,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa",257179,3,Concordo + Concordo totalmente,204797,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"4. Nesta unidade, as pessoas se tratam com respeito",256985,3,Concordo + Concordo totalmente,211661,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes,256448,3,Discordo + Discordo totalmente,140471,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6. Estamos ativamente buscando melhorias para a segurança do paciente,256622,3,Concordo + Concordo totalmente,231369,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente,255779,3,Discordo + Discordo totalmente,166721,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for coluna in df.columns:
    print(coluna, '='*25)
    counts = df[coluna].value_counts()
    percentages = df[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    print('\n')

Ano =========================
      counts percentage
2022   48176      18.6%
2021   43317      16.7%
2020   36945      14.2%
2019   32631      12.6%
2018   27828      10.7%
2017   26698      10.3%
2016   23506       9.1%
2015   13318       5.1%
2014    6849       2.6%


Unidade =========================
               counts percentage
Itaim           13456       5.2%
Copa D'Or       10705       4.1%
Quinta D'Or     10309       4.0%
Brasil           9982       3.9%
Anália Franco    9756       3.8%
...               ...        ...
Yutaka            296       0.1%
Santa Emília      200       0.1%
HNSN Epitácio     198       0.1%
Santa Marina      122       0.0%
Orthoservice       87       0.0%

[71 rows x 2 columns]


Qual é sua principal área ou unidade neste hospital? =========================
                                          counts percentage
Unidade de terapia intensiva adulto        34413      15.3%
Unidade de Internação adulto               31293      13.9%
Administrativa

                                counts percentage
Discordo + Discordo totalmente  121999      47.8%
Nem um nem outro                 76050      29.8%
Concordo + Concordo totalmente   57379      22.5%


39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima =========================
                                counts percentage
Concordo + Concordo totalmente  206676      80.8%
Nem um nem outro                 29702      11.6%
Discordo + Discordo totalmente   19325       7.6%


40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso =========================
                                counts percentage
Discordo + Discordo totalmente  169021      66.1%
Concordo + Concordo totalmente   52484      20.5%
Nem um nem outro                 34361      13.4%


41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes ======================

In [11]:
df_alianca = df[(df['Unidade'] == 'Aliança') & (df['Ano'] == 2022)]

for coluna in df_alianca.columns:
    print(coluna, '='*25)
    counts = df_alianca[coluna].value_counts()
    percentages = df_alianca[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    print('\n')

Ano =========================
      counts percentage
2022    1029     100.0%


Unidade =========================
         counts percentage
Aliança    1029     100.0%


Qual é sua principal área ou unidade neste hospital? =========================
                                          counts percentage
Unidade de Internação adulto                 170      17.4%
Unidade de terapia intensiva adulto          145      14.8%
Emergência adulto                            107      11.0%
Administrativa                                84       8.6%
Centro Cirúrgico                              48       4.9%
Radiologia                                    45       4.6%
Maternidade                                   40       4.1%
Emergência pediátrica                         35       3.6%
Anestesiologia                                32       3.3%
Laboratório Clínico                           29       3.0%
Unidade de terapia intensiva neonatal         29       3.0%
Farmácia                       

In [12]:
lista1 = []
lista2 = []

for coluna in df.columns:
    if 'Nunca + Raramente' in df[coluna].unique():
        lista1.append(coluna)
        #print('alteração 1:', '"' + coluna + '"')
        
        #df[coluna][df[coluna] == 'Maioria do tempo + Sempre'] = 1
        #df[coluna][df[coluna] == 'Algumas vezes'] = 0
        #df[coluna][df[coluna] == 'Nunca + Raramente'] = 0
        #print("\n")
    elif 'Nem um nem outro' in df[coluna].unique():
        lista2.append(coluna)
        #print('alteração 2:', '"' + coluna + '"')
        
        #df[coluna][df[coluna] == 'Concordo + Concordo totalmente'] = 0
        #df[coluna][df[coluna] == 'Nem um nem outro'] = 0
        #df[coluna][df[coluna] == 'Discordo + Discordo totalmente'] = 1
        #print("\n")

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_17556\3991450850.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'Nunca + Raramente' in df[coluna].unique():
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_17556\3991450850.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif 'Nem um nem outro' in df[coluna].unique():


In [13]:
lista1

['23. Recebemos feedback das mudanças implementadas com base nos eventos notificados',
 '24. Os funcionários falam voluntariamente se vêem algo que possa afetar negativamente o atendimento aos pacientes',
 '25. Somos informados sobre os erros que acontecem nesta unidade',
 '26. Os funcionários sentem-se à vontade para questionar decisões ou ações dos que têm mais autoridade',
 '27. Nesta unidade, discutimos maneiras de impedir que os erros tornem a acontecer',
 '28. Os funcionários têm receio de perguntar quando algo não parece certo',
 '29. Quando acontece um erro, mas ele é identificado e corrigido antes de afetar o paciente, com que frequência é notificado?',
 '30. Quando acontece um erro, mas que não tem potencial de dano ao paciente, com que frequência é notificado?',
 '31. Quando acontece um erro que poderia prejudicar o paciente, mas isto não ocorreu, com que frequência é notificado?']

In [14]:
lista1.remove('28. Os funcionários têm receio de perguntar quando algo não parece certo')
for coluna in lista1:
    df[coluna][df[coluna] == 'Maioria do tempo + Sempre'] = 1
    df[coluna][df[coluna] == 'Algumas vezes'] = 0
    df[coluna][df[coluna] == 'Nunca + Raramente'] = 0

coluna = '28. Os funcionários têm receio de perguntar quando algo não parece certo'
df[coluna][df[coluna] == 'Maioria do tempo + Sempre'] = 0
df[coluna][df[coluna] == 'Algumas vezes'] = 0
df[coluna][df[coluna] == 'Nunca + Raramente'] = 1

print('Ok - Continue')

Ok - Continue


In [15]:
lista2

['1. As pessoas se apoiam umas às outras nesta unidade',
 '2. Temos pessoas suficientes para lidar com o volume de trabalho',
 '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
 '4. Nesta unidade, as pessoas se tratam com respeito',
 '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
 '6. Estamos ativamente buscando melhorias para a segurança do paciente',
 '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
 '8. Os funcionários sentem que seus erros são usados contra eles',
 '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
 '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
 '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam',
 '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
 '13. Quando fazemos mudanças para melh

In [16]:
lista_a2 = ['1. As pessoas se apoiam umas às outras nesta unidade',
            '2. Temos pessoas suficientes para lidar com o volume de trabalho',
            '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
            '4. Nesta unidade, as pessoas se tratam com respeito',
            '6. Estamos ativamente buscando melhorias para a segurança do paciente',
            '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
            '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam',
            '13. Quando fazemos mudanças para melhorar a segurança do paciente, nós avaliamos sua efetividade',
            '15. A segurança do paciente nunca é sacrificada em prol de se trabalhar mais',
            '18. Nossos procedimentos e sistemas são bons para impedir que os erros aconteçam',
            '19. Meu supervisor/gerente elogia quando vê um trabalho feito de acordo com os procedimentos estabelecidos para a segurança do paciente.',
            '20. Meu supervisor/gerente considera seriamente as sugestões dos funcionários para melhorar a segurança do paciente',
            '32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente',
            '35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas',
            '39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima',
            '41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes']
for coluna in lista_a2:
    df[coluna][df[coluna] == 'Concordo + Concordo totalmente'] = 1
    df[coluna][df[coluna] == 'Nem um nem outro'] = 0
    df[coluna][df[coluna] == 'Discordo + Discordo totalmente'] = 0
    
print('Ok - Continue')

Ok - Continue


In [17]:
lista_a3 = ['5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
            '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
            '8. Os funcionários sentem que seus erros são usados contra eles',
            '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
            '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
            '14. Trabalhamos “em modo de crise”, tentando fazer coisas demais, rápido demais',
            '16. Os funcionários se preocupam que seus erros sejam registrados em seu arquivo funcional',
            '17. Temos problemas com a segurança do paciente nesta unidade',
            '21. Sempre que a pressão aumenta, meu supervisor/gerente quer que trabalhemos mais rápido, mesmo que isto signifique tomar atalhos',
            '22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente',
            '33. As unidades hospitalares não se coordenam bem entre si.',
            '34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra',
            '36. Importantes informações sobre a assistência se perdem durante as mudanças de turno',
            '37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital',
            '38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital',
            '40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso',
            '42. As mudanças de turno são problemáticas para os pacientes deste Hospital']

for coluna in lista_a3:
    df[coluna][df[coluna] == 'Concordo + Concordo totalmente'] = 0
    df[coluna][df[coluna] == 'Nem um nem outro'] = 0
    df[coluna][df[coluna] == 'Discordo + Discordo totalmente'] = 1
    
print('Ok - Continue')

Ok - Continue


In [18]:
for coluna in df.columns:
    print(coluna, '='*25)
    counts = df[coluna].value_counts()
    percentages = df[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    print('\n')

Ano =========================
      counts percentage
2022   48176      18.6%
2021   43317      16.7%
2020   36945      14.2%
2019   32631      12.6%
2018   27828      10.7%
2017   26698      10.3%
2016   23506       9.1%
2015   13318       5.1%
2014    6849       2.6%


Unidade =========================
               counts percentage
Itaim           13456       5.2%
Copa D'Or       10705       4.1%
Quinta D'Or     10309       4.0%
Brasil           9982       3.9%
Anália Franco    9756       3.8%
...               ...        ...
Yutaka            296       0.1%
Santa Emília      200       0.1%
HNSN Epitácio     198       0.1%
Santa Marina      122       0.0%
Orthoservice       87       0.0%

[71 rows x 2 columns]


Qual é sua principal área ou unidade neste hospital? =========================
                                          counts percentage
Unidade de terapia intensiva adulto        34413      15.3%
Unidade de Internação adulto               31293      13.9%
Administrativa

              counts percentage
Muito boa      93209      36.4%
Excelente      76745      30.0%
Satisfatória   75626      29.6%
Ruim            7805       3.1%
Muito ruim      2515       1.0%


32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente =========================
   counts percentage
1  212043      82.5%
0   45117      17.5%


33. As unidades hospitalares não se coordenam bem entre si. =========================
   counts percentage
1  143241      55.9%
0  113047      44.1%


34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra =========================
   counts percentage
1  136450      53.4%
0  119197      46.6%


35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas =========================
   counts percentage
1  158040      61.7%
0   98110      38.3%


36. Importantes informações sobre a assistência se perdem durante as mudanças de turno ====

In [19]:
lista = ['1. As pessoas se apoiam umas às outras nesta unidade',
       '2. Temos pessoas suficientes para lidar com o volume de trabalho',
       '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
       '4. Nesta unidade, as pessoas se tratam com respeito',
       '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
       '6. Estamos ativamente buscando melhorias para a segurança do paciente',
       '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
       '8. Os funcionários sentem que seus erros são usados contra eles',
       '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
       '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
       '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam',
       '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
       '13. Quando fazemos mudanças para melhorar a segurança do paciente, nós avaliamos sua efetividade',
       '14. Trabalhamos “em modo de crise”, tentando fazer coisas demais, rápido demais',
       '15. A segurança do paciente nunca é sacrificada em prol de se trabalhar mais',
       '16. Os funcionários se preocupam que seus erros sejam registrados em seu arquivo funcional',
       '17. Temos problemas com a segurança do paciente nesta unidade',
       '18. Nossos procedimentos e sistemas são bons para impedir que os erros aconteçam',
       '19. Meu supervisor/gerente elogia quando vê um trabalho feito de acordo com os procedimentos estabelecidos para a segurança do paciente.',
       '20. Meu supervisor/gerente considera seriamente as sugestões dos funcionários para melhorar a segurança do paciente',
       '21. Sempre que a pressão aumenta, meu supervisor/gerente quer que trabalhemos mais rápido, mesmo que isto signifique tomar atalhos',
       '22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente',
       '23. Recebemos feedback das mudanças implementadas com base nos eventos notificados',
       '24. Os funcionários falam voluntariamente se vêem algo que possa afetar negativamente o atendimento aos pacientes',
       '25. Somos informados sobre os erros que acontecem nesta unidade',
       '26. Os funcionários sentem-se à vontade para questionar decisões ou ações dos que têm mais autoridade',
       '27. Nesta unidade, discutimos maneiras de impedir que os erros tornem a acontecer',
       '28. Os funcionários têm receio de perguntar quando algo não parece certo',
       '29. Quando acontece um erro, mas ele é identificado e corrigido antes de afetar o paciente, com que frequência é notificado?',
       '30. Quando acontece um erro, mas que não tem potencial de dano ao paciente, com que frequência é notificado?',
       '31. Quando acontece um erro que poderia prejudicar o paciente, mas isto não ocorreu, com que frequência é notificado?',
       'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar.',
       '32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente',
       '33. As unidades hospitalares não se coordenam bem entre si.',
       '34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra',
       '35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas',
       '36. Importantes informações sobre a assistência se perdem durante as mudanças de turno',
       '37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital',
       '38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital',
       '39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima',
       '40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso',
       '41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes',
       '42. As mudanças de turno são problemáticas para os pacientes deste Hospital']

# verificando as proporções:

In [20]:
df_backup1 = df_backup[(df_backup['Ano'] == 2022) & (df_backup['Unidade'] == 'Aliança')].reset_index(drop=True)
df1 = df[(df['Ano'] == 2022) & (df['Unidade'] == 'Anália Franco')].reset_index(drop=True)

for coluna in lista:
    print(coluna, '='*50)
    
    counts = df_backup1[coluna].value_counts()
    percentages = df_backup1[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    
    print('\n')
    #
    
    counts = df1[coluna].value_counts()
    percentages = df1[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    
    print('\n')

1. As pessoas se apoiam umas às outras nesta unidade ==================================================
                                counts percentage
Concordo + Concordo totalmente     858      83.5%
Nem um nem outro                    85       8.3%
Discordo + Discordo totalmente      84       8.2%


   counts percentage
1     742      65.7%
0     388      34.3%


2. Temos pessoas suficientes para lidar com o volume de trabalho ==================================================
                                counts percentage
Concordo + Concordo totalmente     457      44.5%
Discordo + Discordo totalmente     428      41.6%
Nem um nem outro                   143      13.9%


   counts percentage
0     814      72.2%
1     314      27.8%


3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa ==================================================
                                counts percentage
Concordo + Concordo totalmente     853

                                counts percentage
Discordo + Discordo totalmente     652      63.8%
Nem um nem outro                   198      19.4%
Concordo + Concordo totalmente     172      16.8%


   counts percentage
0     651      57.7%
1     477      42.3%


35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas ==================================================
                                counts percentage
Concordo + Concordo totalmente     697      68.3%
Nem um nem outro                   195      19.1%
Discordo + Discordo totalmente     129      12.6%


   counts percentage
0     588      52.2%
1     538      47.8%


36. Importantes informações sobre a assistência se perdem durante as mudanças de turno ==================================================
                                counts percentage
Discordo + Discordo totalmente     729      71.5%
Nem um nem outro                   145      14.2%
Concordo + Concordo totalmente     1

In [21]:
for i in df.columns:
    print(i, '='*75)
    print(df[i].unique())
    print('\n')

Ano ===========================================================================
[2014 2015 2016 2017 2018 2019 2020 2021 2022]


Unidade ===========================================================================
['Coração do Brasil' 'Santa Luzia' 'Esperança Olinda' 'São Marcos'
 'Esperança' 'Memorial São José' 'Santa Helena' 'UDI' 'DF Star'
 'São Lucas' 'São Rafael' 'Aliança' 'Cárdio Pulmonar' 'São Carlos'
 '5 de outubro' 'Proncor' 'Santa Marina' 'Yutaka' 'Aeroporto'
 'Arthur Ramos' 'HNSN Epitácio' 'HNSN JB' 'Santa Emília' "Barra D'Or"
 "Caxias D'Or" 'Criança RJ' "Niterói D'Or" "Norte D'Or" "Oeste D'Or"
 "Quinta D'Or" "Rios D'Or" 'Bangu' "Copa D'Or" 'Copa Star' "Real D'Or"
 'São Vicente' 'Jutta Batista' "Glória D'Or" 'Perinatal Barra'
 'Perinatal Laranjeiras' 'Rio Barra' 'Samer' 'Balbino' 'São Lucas Macaé'
 'Anália Franco' 'Assunção' 'Itaim' 'Brasil' 'Criança SP' 'Jabaquara'
 'Morumbi' 'Bartira' 'IFOR' 'Villa Lobos' 'viValle' 'Osasco'
 'Ribeirão Pires' 'Alpha Med' 'Jabaquara/ Criança'

[0 1 nan]


39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima ===========================================================================
[1 0 nan]


40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso ===========================================================================
[0 1 nan]


41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes ===========================================================================
[0 1 nan]


42. As mudanças de turno são problemáticas para os pacientes deste Hospital ===========================================================================
[0 1 nan]


SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou? ===========================================================================
['a. Nenhuma notificações de eventos'
 'f. 21 

In [22]:
coluna = 'SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?'

print(df[coluna].value_counts(), '\n')
df['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses'] = 0
df['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses'][df[coluna] != 'a. Nenhuma notificações de eventos'] = 1
df['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses'][pd.isnull(df[coluna])] = np.nan
print(df['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses'].value_counts(),'\n')

#################################################################################################
df[coluna][df[coluna] == 'a. Nenhuma notificações de eventos'] = '0. Nenhuma notificações de eventos'
df[coluna][df[coluna] == 'b. 1 a 2 notificações de eventos'] = '1. 1 a 2 notificações de eventos'
df[coluna][df[coluna] == 'c. 3 a 5 notificações de eventos'] = '2. 3 a 5 notificações de eventos'
df[coluna][df[coluna] == 'd. 6 a 10 notificações de eventos'] = '3. 6 a 10 notificações de eventos'
df[coluna][df[coluna] == 'e. 11 a 20 notificações de eventos'] = '4. 11 a 20 notificações de eventos'
df[coluna][df[coluna] == 'f. 21 notificações de eventos ou mais'] = '5. 21 notificações de eventos ou mais'
print(df[coluna].value_counts())

# Gerando as variáveis dummy
dummies = pd.get_dummies(df[coluna], prefix=coluna)
df = pd.concat([df, dummies], axis=1)

a. Nenhuma notificações de eventos       120207
b. 1 a 2 notificações de eventos          55183
c. 3 a 5 notificações de eventos          36186
d. 6 a 10 notificações de eventos         22258
e. 11 a 20 notificações de eventos        11726
f. 21 notificações de eventos ou mais     10597
Name: SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?, dtype: int64 

1.0    135950
0.0    120207
Name: SEÇÃO G2: Notificou algum evento nos ultimos 12 meses, dtype: int64 

0. Nenhuma notificações de eventos       120207
1. 1 a 2 notificações de eventos          55183
2. 3 a 5 notificações de eventos          36186
3. 6 a 10 notificações de eventos         22258
4. 11 a 20 notificações de eventos        11726
5. 21 notificações de eventos ou mais     10597
Name: SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?, dtype: int64


In [23]:
coluna = 'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'
df[coluna].unique()

array(['1 a 5 anos', 'menos de 1 ano', '6 a 10 anos', nan, '11 a 15 anos',
       '16 a 20 anos', '21 anos ou mais'], dtype=object)

In [24]:
#Na unidade tem muito funcionario que sobrecarregado ?
coluna = 'SEÇÃO H: 3.Tipicamente, quantas horas por semana você trabalha neste hospital?  '
print(df[coluna].value_counts(),'\n')

df['SEÇÃO H: 3.2 Trabalha mais que 40horas semanais'] = 1
df['SEÇÃO H: 3.2 Trabalha mais que 40horas semanais'][(df[coluna] == 'a. Menos de 20 horas por semana') |
                                                        (df[coluna] == 'b. 20 a 39 horas por semana')] = 0
df['SEÇÃO H: 3.2 Trabalha mais que 40horas semanais'][pd.isnull(df[coluna])] = np.nan
print(df['SEÇÃO H: 3.2 Trabalha mais que 40horas semanais'].value_counts(), '\n')

##########################################################################
df['SEÇÃO H: 3.2 Trabalha mais que 60horas semanais'] = 1
df['SEÇÃO H: 3.2 Trabalha mais que 60horas semanais'][(df[coluna] == 'a. Menos de 20 horas por semana') |
                                                      (df[coluna] == 'b. 20 a 39 horas por semana') |
                                                      (df[coluna] == 'c. 40 a 59 horas por semana')] = 0
df['SEÇÃO H: 3.2 Trabalha mais que 60horas semanais'][pd.isnull(df[coluna])] = np.nan
print(df['SEÇÃO H: 3.2 Trabalha mais que 60horas semanais'].value_counts(), '\n')

##########################################################################
df['SEÇÃO H: 3.2 Trabalha mais que 80horas semanais'] = 0
df['SEÇÃO H: 3.2 Trabalha mais que 80horas semanais'][(df[coluna] == 'e. 80 a 99 horas por semana') |
                                                      (df[coluna] == 'f. 100 horas por semana ou mais')] = 1
df['SEÇÃO H: 3.2 Trabalha mais que 80horas semanais'][pd.isnull(df[coluna])] = np.nan
print(df['SEÇÃO H: 3.2 Trabalha mais que 80horas semanais'].value_counts(), '\n')

##########################################################################
coluna2 = 'SEÇÃO H: Horas trabalhadas semanais'
df[coluna2] = np.nan
df[coluna2][df[coluna] == 'a. Menos de 20 horas por semana'] = '0. 20 a 39'
df[coluna2][df[coluna] == 'b. 20 a 39 horas por semana'] = '0. 20 a 39'
df[coluna2][df[coluna] == 'c. 40 a 59 horas por semana'] = '1. 40 a - 79'
df[coluna2][df[coluna] == 'd. 60 a 79 horas por semana'] = '1. 40 a - 79'
df[coluna2][df[coluna] == 'e. 80 a 99 horas por semana'] = '2. > 80'
df[coluna2][df[coluna] == 'f. 100 horas por semana ou mais'] = '2. > 80'

########################################################################## ARRUMANDO OS NIVEIS
coluna = "SEÇÃO H: 3.Tipicamente, quantas horas por semana você trabalha neste hospital?  "
df[coluna][df[coluna] == 'a. Menos de 20 horas por semana'] = '1. Menos de 20 horas por semana'
df[coluna][df[coluna] == 'b. 20 a 39 horas por semana'] = '2. 20 a 39 horas por semana'
df[coluna][df[coluna] == 'c. 40 a 59 horas por semana'] = '3. 40 a 59 horas por semana'
df[coluna][df[coluna] == 'd. 60 a 79 horas por semana'] = '4. 60 a 79 horas por semana'
df[coluna][df[coluna] == 'e. 80 a 99 horas por semana'] = '5. 80 a 99 horas por semana'
df[coluna][df[coluna] == 'f. 100 horas por semana ou mais'] = '6. 100 horas por semana ou mais'

c. 40 a 59 horas por semana        135076
b. 20 a 39 horas por semana         91281
a. Menos de 20 horas por semana     18421
d. 60 a 79 horas por semana          8078
e. 80 a 99 horas por semana          1833
f. 100 horas por semana ou mais      1641
Name: SEÇÃO H: 3.Tipicamente, quantas horas por semana você trabalha neste hospital?  , dtype: int64 

1.0    146628
0.0    109702
Name: SEÇÃO H: 3.2 Trabalha mais que 40horas semanais, dtype: int64 

0.0    244778
1.0     11552
Name: SEÇÃO H: 3.2 Trabalha mais que 60horas semanais, dtype: int64 

0.0    252856
1.0      3474
Name: SEÇÃO H: 3.2 Trabalha mais que 80horas semanais, dtype: int64 



In [25]:
coluna = 'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar.'
print(df[coluna].value_counts(),'\n')

# Gerando as variáveis dummy
dummies = pd.get_dummies(df[coluna], prefix=coluna)
df = pd.concat([df, dummies], axis=1)

df['Nota Geral == Satisfatoria ?'] = 1
df['Nota Geral == Satisfatoria ?'][(df[coluna] == 'Ruim') |
                                   (df[coluna] == 'Muito ruim')] = 0
df['Nota Geral == Satisfatoria ?'][pd.isnull(df[coluna])] = np.nan
print(df['Nota Geral == Satisfatoria ?'].value_counts(), '\n')

##########################################################################
df['Nota Geral == Boa ?'] = 1
df['Nota Geral == Boa ?'][(df[coluna] == 'Ruim') |
                          (df[coluna] == 'Muito ruim') |
                         (df[coluna] == 'Satisfatória')] = 0
df['Nota Geral == Boa ?'][pd.isnull(df[coluna])] = np.nan
print(df['Nota Geral == Boa ?'].value_counts(), '\n')

##########################################################################
df['Nota Geral'] = 1
df['Nota Geral'][(df[coluna] == 'Ruim') |
                   (df[coluna] == 'Muito ruim')] = 0
df['Nota Geral'][df[coluna] == 'Satisfatória'] = 1
df['Nota Geral'][(df[coluna] == 'Excelente') |
                   (df[coluna] == 'Muito boa')] = 2
df['Nota Geral'][pd.isnull(df[coluna])] = np.nan
print(df['Nota Geral'].value_counts(), '\n')

########################################################################## ARRUMANDO NIVEIS
df[coluna][df[coluna] == 'Muito ruim'] = '0. Muito ruim'
df[coluna][df[coluna] == 'Ruim'] = '1. Ruim'
df[coluna][df[coluna] == 'Satisfatória'] = '2. Satisfatória'
df[coluna][df[coluna] == 'Excelente'] = '3. Excelente'
df[coluna][df[coluna] == 'Muito boa'] = '4. Muito boa'
print(df[coluna].value_counts(),'\n')

Muito boa       93209
Excelente       76745
Satisfatória    75626
Ruim             7805
Muito ruim       2515
Name: Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar., dtype: int64 

1.0    245580
0.0     10320
Name: Nota Geral == Satisfatoria ?, dtype: int64 

1.0    169954
0.0     85946
Name: Nota Geral == Boa ?, dtype: int64 

2.0    169954
1.0     75626
0.0     10320
Name: Nota Geral, dtype: int64 

4. Muito boa       93209
3. Excelente       76745
2. Satisfatória    75626
1. Ruim             7805
0. Muito ruim       2515
Name: Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar., dtype: int64 



In [26]:
df.columns

Index(['Ano', 'Unidade',
       'Qual é sua principal área ou unidade neste hospital?',
       '1. As pessoas se apoiam umas às outras nesta unidade',
       '2. Temos pessoas suficientes para lidar com o volume de trabalho',
       '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
       '4. Nesta unidade, as pessoas se tratam com respeito',
       '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
       '6. Estamos ativamente buscando melhorias para a segurança do paciente',
       '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
       '8. Os funcionários sentem que seus erros são usados contra eles',
       '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
       '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
       '11. Quando uma área nesta unidade fica muito movimentada, as demai

In [27]:
print(df['Qual é sua principal área ou unidade neste hospital?'].value_counts(),'\n')

df['Setor'] = np.nan
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Administrativa'] = 'Gestão'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Anestesiologia'] = 'Processo Cirurgico ou Obstétrico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Cardiologia Intervencionistas'] = 'Processo Cirurgico ou Obstétrico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Central de Material Esterilizado'] = 'Área de Apoio'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Centro Cirúrgico'] = 'Processo Cirurgico ou Obstétrico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Centro Obstétrico'] = 'Processo Cirurgico ou Obstétrico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Diretoria'] = 'Gestão'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Emergência adulto'] = 'Emergência'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Emergência ginecologia'] = 'Emergência'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Emergência pediátrica'] = 'Emergência'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Endoscopia'] = 'Processo Cirurgico ou Obstétrico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Farmácia'] = 'Área de Apoio'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Gerência'] = 'Área de Apoio'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Laboratório Clínico'] = 'Serviço Diagnóstico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Maternidade'] = 'Processo Cirurgico ou Obstétrico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Nutrição'] = 'Área de Apoio'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Pediatria'] = 'UI'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Psiquiatria/Saúde mental'] = 'UI'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Radiologia'] = 'Serviço Diagnóstico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Reabilitação'] = 'Área de Apoio'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Ressonância Magnética'] = 'Serviço Diagnóstico'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Suprimentos'] = 'Área de Apoio'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Ultrassonografia'] = 'Área de Apoio'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Unidade de Internação adulto'] = 'UI'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Unidade de Internação neonatal/ berçário'] = 'UI'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Unidade de Internação pediátrica'] = 'UI'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Unidade de terapia intensiva adulto'] = 'UTI'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Unidade de terapia intensiva neonatal'] = 'UTI'
df['Setor'][df['Qual é sua principal área ou unidade neste hospital?'] == 'Unidade de terapia intensiva pediátrica'] = 'UTI'

print(df['Setor'].value_counts(), '\n')

Unidade de terapia intensiva adulto         34413
Unidade de Internação adulto                31293
Administrativa                              26157
Emergência adulto                           22037
Centro Cirúrgico                            15104
Farmácia                                    12501
Nutrição                                    11042
Radiologia                                   8225
Laboratório Clínico                          6756
Emergência pediátrica                        5855
Unidade de terapia intensiva pediátrica      5422
Central de Material Esterilizado             5145
Unidade de terapia intensiva neonatal        4984
Maternidade                                  4916
Suprimentos                                  4404
Anestesiologia                               3705
Pediatria                                    3433
Unidade de Internação pediátrica             3295
Reabilitação                                 2318
Cardiologia Intervencionistas                2225


In [28]:
print(df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'].value_counts(), '\n')

#Verificar a rotatividade do hospital
df['SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?'] = 0
df['SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?'][df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'] != 'menos de 1 ano'] = 1
df['SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?'][pd.isnull(df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'])] = np.nan
print(df['SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?'].value_counts(), '\n')

df['SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?'] = 1
df['SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?'][(df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'] == 'menos de 1 ano') | 
                                                          (df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'] == '1 a 5 anos')] = 0
df['SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?'][pd.isnull(df['SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'])] = np.nan
print(df['SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?'].value_counts())


1 a 5 anos         121452
menos de 1 ano      54735
6 a 10 anos         49508
11 a 15 anos        16889
16 a 20 anos         8246
21 anos ou mais      6200
Name: SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?, dtype: int64 

1.0    202295
0.0     54735
Name: SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?, dtype: int64 

0.0    176187
1.0     80843
Name: SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?, dtype: int64


In [29]:
colunas = ["SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?",
           'SEÇÃO H: 2.Há quanto tempo você trabalha nesta área/unidade de trabalho hospitalar?',
           "6. Há quanto tempo você trabalha em sua profissão ou especialização atual?  "]

for coluna in colunas:
    print('\n')
    df[coluna][df[coluna] == 'a. Menos de 1 ano'] = '0. Menos de 1 ano'
    df[coluna][df[coluna] == 'Menos de 1 ano'] = '0. Menos de 1 ano'
    df[coluna][df[coluna] == 'menos de 1 ano'] = '0. Menos de 1 ano'
    
    df[coluna][df[coluna] == 'b. 1 a 5 anos'] = '1. 1 a 5 anos'
    df[coluna][df[coluna] == '1 a 5 anos'] = '1. 1 a 5 anos'
    
    df[coluna][df[coluna] == 'c. 6 a 10 anos'] = '2. 6 a 10 anos'
    df[coluna][df[coluna] == '6 a 10 anos'] = '2. 6 a 10 anos'
    
    df[coluna][df[coluna] == 'd. 11 a 15 anos'] = '3. 11 a 15 anos'
    df[coluna][df[coluna] == '11 a 15 anos'] = '3. 11 a 15 anos'
    
    df[coluna][df[coluna] == 'e. 16 a 20 anos'] = '4. 16 a 20 anos'
    df[coluna][df[coluna] == '16 a 20 anos'] = '4. 16 a 20 anos'
    
    df[coluna][df[coluna] == 'f. 21 anos ou mais'] = '5. 21 anos ou mais'
    df[coluna][df[coluna] == '21 anos ou mais'] = '5. 21 anos ou mais'
    
    print(df[coluna].value_counts())



1. 1 a 5 anos         121452
0. Menos de 1 ano      54735
2. 6 a 10 anos         49508
3. 11 a 15 anos        16889
4. 16 a 20 anos         8246
5. 21 anos ou mais      6200
Name: SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?, dtype: int64


1. 1 a 5 anos         121377
0. Menos de 1 ano      53896
2. 6 a 10 anos         48277
3. 11 a 15 anos        17711
4. 16 a 20 anos         8972
5. 21 anos ou mais      6671
Name: SEÇÃO H: 2.Há quanto tempo você trabalha nesta área/unidade de trabalho hospitalar?, dtype: int64


1. 1 a 5 anos         88963
2. 6 a 10 anos        66117
3. 11 a 15 anos       38371
0. Menos de 1 ano     22232
4. 16 a 20 anos       21939
5. 21 anos ou mais    19523
Name: 6. Há quanto tempo você trabalha em sua profissão ou especialização atual?  , dtype: int64


In [30]:
coluna = 'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?'
# Gerando as variáveis dummy
dummies = pd.get_dummies(df[coluna], prefix=coluna)
df = pd.concat([df, dummies], axis=1)

In [31]:
df.columns

Index(['Ano', 'Unidade',
       'Qual é sua principal área ou unidade neste hospital?',
       '1. As pessoas se apoiam umas às outras nesta unidade',
       '2. Temos pessoas suficientes para lidar com o volume de trabalho',
       '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
       '4. Nesta unidade, as pessoas se tratam com respeito',
       '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
       '6. Estamos ativamente buscando melhorias para a segurança do paciente',
       '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
       '8. Os funcionários sentem que seus erros são usados contra eles',
       '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
       '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
       '11. Quando uma área nesta unidade fica muito movimentada, as demai

In [32]:
df.columns = [col.strip() for col in df.columns]

coluna = "6. Há quanto tempo você trabalha em sua profissão ou especialização atual?"
# Gerando as variáveis dummy
dummies = pd.get_dummies(df[coluna], prefix=coluna)
df = pd.concat([df, dummies], axis=1)

In [33]:
df.to_excel('0 - Dados Limpos.xlsx', index=False)
df_bruto = df.copy()
len(df)

259268

In [34]:
print('quantidade de linhas:',len(df))
print('quantidade de coluna:',len(df.columns))
print('quantidade de dados:',len(df)*len(df.columns))

quantidade de linhas: 259268
quantidade de coluna: 92
quantidade de dados: 23852656


In [35]:
stop

NameError: name 'stop' is not defined

# TESTES / ANALISES =================================

In [ ]:
for i in lista:
    print('coluna:', i)
    print('0: mean(', round(resultado[i][resultado['acreditacao'] == 0].mean(),4), ') ; median(', round(resultado[i][resultado['acreditacao'] == 0].median(),4), ')')
    print('1: mean(', round(resultado[i][resultado['acreditacao'] == 1].mean(),4), ') ; median(', round(resultado[i][resultado['acreditacao'] == 1].median(),4), ')')
    
    a = resultado[i][resultado['acreditacao'] == 0] # amostra a
    b = resultado[i][resultado['acreditacao'] == 1] # amostra b
    ttest = round(stats.ttest_ind(a,b)[1], 4)
    mann = round(stats.mannwhitneyu(a, b)[1], 4)
    print('pvalor:  '+str(ttest)+'             '+str(mann))
    
    grupos = resultado['acreditacao'].unique()

    plt.figure(figsize=(10,2))

    for grupo in grupos:

        if grupo == 0:
            label_grupo = "Sem Acreditação"
        else:
            label_grupo = "Com Acreditação"

        sns.kdeplot(resultado[resultado['acreditacao'] == grupo][i], label=label_grupo)

    plt.axvline(x = resultado[i][resultado['acreditacao'] == 0].mean(), color='red', linestyle='--', label='Média sem')
    plt.axvline(x = resultado[i][resultado['acreditacao'] == 0].median(), color='blue', linestyle='--', label='Mediana sem')

    plt.axvline(x = resultado[i][resultado['acreditacao'] == 1].mean(), color='orange', linestyle='--', label='Média com')
    plt.axvline(x = resultado[i][resultado['acreditacao'] == 1].median(), color='brown', linestyle='--', label='Mediana com')

    plt.title(i)
    plt.legend()
    plt.show()
    
    print('='*125)

# APLICANDO ANOVA:

In [ ]:
resultado1 = resultado.rename(columns={"42. As mudanças de turno são problemáticas para os pacientes deste Hospital": "var_num"})

# Ajuste o modelo
modelo = ols('var_num ~ C(acreditacao) + C(Regional) + C(acreditacao):C(Regional)', data=resultado1).fit()

# Realize a ANOVA e imprima a tabela de resultados
anova_table = sm.stats.anova_lm(modelo, typ=2)
print(anova_table)


In [ ]:
modelo = ols('var_num ~ C(acreditacao)', data = resultado1).fit()

# Realize a ANOVA e imprima a tabela de resultados
anova_table = sm.stats.anova_lm(modelo, typ=2)
print(anova_table)

In [ ]:
resultado['Regional2'] = np.nan
resultado['Regional2'][(resultado['Regional'] == 'SUL') | (resultado['Regional'] == 'SE')] = 'SUL'
resultado['Regional2'][(resultado['Regional'] == 'N') | (resultado['Regional'] == 'NE')] = 'NORTE'
resultado['Regional2'][resultado['Regional'] == 'CO'] = 'CO'
print(resultado['Regional2'].value_counts())

##########
resultado['Regional3'] = resultado['Regional2']
resultado['Regional3'][(resultado['Regional2'] == 'CO') | (resultado['Regional2'] == 'NORTE')] = 'NORTE_OC'
print(resultado['Regional3'].value_counts())

In [ ]:
#resultado.to_excel('tabela3.xlsx')

In [ ]:
for i in lista:
    print('coluna:', i)
    print('NORTE_OC: mean(', round(resultado[i][resultado['Regional3'] == 'NORTE_OC'].mean(),4), ') ; median(', round(resultado[i][resultado['Regional3'] == 'NORTE_OC'].median(),4), ')')
    print('SUL: mean(', round(resultado[i][resultado['Regional3'] == 'SUL'].mean(),4), ') ; median(', round(resultado[i][resultado['Regional3'] == 'SUL'].median(),4), ')')
    
    a = resultado[i][resultado['Regional3'] == 'NORTE_OC'] # amostra a
    b = resultado[i][resultado['Regional3'] == 'SUL'] # amostra b
    ttest = round(stats.ttest_ind(a,b)[1], 4)
    mann = round(stats.mannwhitneyu(a, b)[1], 4)
    print('pvalor:  '+str(ttest)+'             '+str(mann))
    
    grupos = resultado['Regional3'].unique()

    plt.figure(figsize=(10,2))

    for grupo in grupos:

        if grupo == 'NORTE_OC':
            label_grupo = "NORTE"
        else:
            label_grupo = "SUL"

        sns.kdeplot(resultado[resultado['Regional3'] == grupo][i], label=label_grupo)

    plt.axvline(x = resultado[i][resultado['Regional3'] == 'NORTE_OC'].mean(), color='red', linestyle='--', label='Média NORTE')
    plt.axvline(x = resultado[i][resultado['Regional3'] == 'NORTE_OC'].median(), color='blue', linestyle='--', label='Mediana NORTE')

    plt.axvline(x = resultado[i][resultado['Regional3'] == 'SUL'].mean(), color='orange', linestyle='--', label='Média SUL')
    plt.axvline(x = resultado[i][resultado['Regional3'] == 'SUL'].median(), color='brown', linestyle='--', label='Mediana SUL')

    plt.title(i)
    plt.legend()
    plt.show()
    
    print('='*125)

In [ ]:
df

In [ ]:
df = pd.read_excel("tabela3.xlsx")
print(len(df))
df.sample(5)

# VERFICANDO SE HOUVE MELHORA

In [ ]:
for i in lista:
    print('coluna:', i)
    print('0: mean(', round(tabela[i][tabela['acreditacao'] == 0].mean(),4), ') ; median(', round(tabela[i][tabela['acreditacao'] == 0].median(),4), ')')
    print('1: mean(', round(tabela[i][tabela['acreditacao'] == 1].mean(),4), ') ; median(', round(tabela[i][tabela['acreditacao'] == 1].median(),4), ')')
    
    a = tabela[i][tabela['acreditacao'] == 0] # amostra a
    b = tabela[i][tabela['acreditacao'] == 1] # amostra b
    ttest = round(stats.ttest_ind(a,b)[1], 4)
    mann = round(stats.mannwhitneyu(a, b)[1], 4)
    print('pvalor:  '+str(ttest)+'             '+str(mann))
    
    grupos = tabela['acreditacao'].unique()

    plt.figure(figsize=(10,2))

    for grupo in grupos:

        if grupo == 0:
            label_grupo = "Sem Acreditação"
        else:
            label_grupo = "Com Acreditação"

        sns.kdeplot(tabela[tabela['acreditacao'] == grupo][i], label=label_grupo)

    plt.axvline(x = tabela[i][tabela['acreditacao'] == 0].mean(), color='red', linestyle='--', label='Média sem')
    plt.axvline(x = tabela[i][tabela['acreditacao'] == 0].median(), color='blue', linestyle='--', label='Mediana sem')

    plt.axvline(x = tabela[i][tabela['acreditacao'] == 1].mean(), color='orange', linestyle='--', label='Média com')
    plt.axvline(x = tabela[i][tabela['acreditacao'] == 1].median(), color='brown', linestyle='--', label='Mediana com')

    plt.title(i)
    plt.legend()
    plt.show()
    
    print('='*125)

In [ ]:
count = 0
for unidade in sorted(df['Unidade'].unique()):
    if len(df['acreditacao'][df['Unidade'] == unidade].unique()) == 1:
        if df['acreditacao'][df['Unidade'] == unidade].unique()[0] == 0:
            print(unidade, '='*25)
            print(df[df['Unidade'] == unidade].groupby(['Ano', 'acreditacao']).size(), '\n')
            count += 1
            
print('='*25, 'QUANTIDADE:', count, '='*25)

In [ ]:
count = 0 
for unidade in sorted(df['Unidade'].unique()):
    if len(df['acreditacao'][df['Unidade'] == unidade].unique()) == 1:
        if df['acreditacao'][df['Unidade'] == unidade].unique()[0] == 1:
            print(unidade, '='*25)
            print(df[df['Unidade'] == unidade].groupby(['Ano', 'acreditacao']).size(), '\n')
            count += 1
            
print('='*25, 'QUANTIDADE:', count, '='*25)

# Existe uma mudança no score com o passar do tempo

In [ ]:
#tabela = df[df['acreditacao'] == 1].reset_index(drop=True)
#tabela = df
count = 0 
for coluna in lista:
    subtitulo = ''
    mediana = df.groupby(['Ano', 'acreditacao'])[coluna].median().unstack().reset_index()
    
    ##############################################################################################
    for acre in range(0, 2):
        X = df['Ano'][df['acreditacao'] == acre]
        Y = df[coluna][df['acreditacao'] == acre]
        X = sm.add_constant(X)
        model = sm.OLS(Y, X)
        results = model.fit()
        p_valor = round(results.pvalues['Ano'], 3)
        if p_valor < 0.001:p_valor="<0.001"
        coef = str(round(results.params['Ano'], 2))
        interval = results.conf_int(alpha=0.05).loc['Ano']
        interval_0 = str(round(interval[0], 2))
        interval_1 = str(round(interval[1], 2))
        subtitulo = coef + ' (' + interval_0 + ' - ' + interval_1 + ')'
        subtitulo = 'p-valor = ' + str(p_valor) + ' | '+ subtitulo
        subtitulo = subtitulo + " | R² = " + str(round(results.rsquared, 2))
        
        subtitulo = str(round(Y.mean(), 2)) + " | " + subtitulo

        plt.figure(figsize=(10,3))
        if acre == 0:
            cor = "red"
            plt.title(coluna, fontproperties=fonte, fontsize=16)
        else:
            cor = 'blue'
        sns.scatterplot(x="Ano", y=coluna, data=df[df['acreditacao'] == acre],
                           edgecolor = "black", color = cor)
        sns.lineplot(x='Ano', y=acre, data = mediana, color = 'green')
        sns.lineplot(x="Ano", y=coluna, data=df[df['acreditacao'] == acre], color = 'red')

        plt.xlabel(subtitulo, fontproperties=fonte, fontsize=13)
        plt.ylabel(str(acre)+'acreditação', fontproperties=fonte, fontsize=13)
        plt.ylim(0, 100)
        plt.show()

        ##############################################################################################

In [ ]:
#tabela = df[df['acreditacao'] == 1].reset_index(drop=True)
#tabela = df
count = 0 
for coluna in lista:
    subtituloo = ''
    plt.figure(figsize=(10,3))
    sns.lineplot(x="Ano", y=coluna, hue='acreditacao', data=df)
    plt.title(coluna, fontproperties=fonte, fontsize=16)
    plt.ylabel('Score Médio', fontproperties=fonte, fontsize=13)
    #plt.ylim(0, 100)
    
    ##############################################################################################
    for acre in range(0, 2):
        X = df['Ano'][df['acreditacao'] == acre]
        Y = df[coluna][df['acreditacao'] == acre]
        X = sm.add_constant(X)
        model = sm.OLS(Y, X)
        results = model.fit()
        p_valor = round(results.pvalues['Ano'], 3)
        if p_valor < 0.001:p_valor="<0.001"
        coef = str(round(results.params['Ano'], 2))
        interval = results.conf_int(alpha=0.05).loc['Ano']
        interval_0 = str(round(interval[0], 2))
        interval_1 = str(round(interval[1], 2))
        subtitulo = coef + ' (' + interval_0 + ' - ' + interval_1 + ')'
        subtitulo = 'p-valor = ' + str(p_valor) + ' | '+ subtitulo
        subtitulo = subtitulo + " | R² = " + str(round(results.rsquared, 2))
        
        subtitulo = str(round(Y.mean(), 2)) + " | " + subtitulo
        subtitulo = str(acre) + "=> " + subtitulo
        subtitulo = subtitulo + " | " + str(round(durbin_watson(results.resid), 2))
        
        if acre == 1:
            subtituloo = subtituloo + '\n' + subtitulo
        else:
            subtituloo = subtituloo + subtitulo
    
    plt.xlabel(subtituloo, fontproperties=fonte, fontsize=13)
    plt.show()

        ##############################################################################################

In [ ]:
durbinWatson = durbin_watson(results.resid)
round(durbinWatson, 2)

In [ ]:


for i in lista:
    print('='*25, i, '='*25)
    Y = df[i]
    X = df[['Ano', 'acreditacao']]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    results = model.fit()
    print(results.summary())
    durbinWatson = durbin_watson(results.resid)
    print('\n')
    print('O valor de Durbin-Watson é:', durbinWatson)
    print('\n')

In [ ]:
df

In [ ]:
df

In [ ]:
lista2 = ['Abertura de comunicação','Feedback e comunicação sobre erros',
          'Frequência de eventos relatados','Apoio de gestão hospitalar para segurança do paciente',
          'Trabalho em equipe entre as unidades hospitalares','Trabalho em equipe no âmbito das unidades',
          'Transferências internas e passagens de plantão','Aprendizado organizacional - melhoria contínua',
          'Expectativas de promoção da segurança dos supervisores e gerentes','Percepção gerais sobre segurança',
          'Respostas não punitivas aos erros','Staffing']
count = 0 
for coluna in lista2:
    subtituloo = ''
    plt.figure(figsize=(10,3))
    sns.lineplot(x="Ano", y=coluna, hue='acreditacao', data=df)
    plt.title(coluna, fontproperties=fonte, fontsize=16)
    plt.ylabel('Score Médio', fontproperties=fonte, fontsize=13)
    #plt.ylim(0, 100)
    
    ##############################################################################################
    for acre in range(0, 2):
        X = df['Ano'][df['acreditacao'] == acre]
        Y = df[coluna][df['acreditacao'] == acre]
        X = sm.add_constant(X)
        model = sm.OLS(Y, X)
        results = model.fit()
        p_valor = round(results.pvalues['Ano'], 3)
        if p_valor < 0.001:p_valor="<0.001"
        coef = str(round(results.params['Ano'], 2))
        interval = results.conf_int(alpha=0.05).loc['Ano']
        interval_0 = str(round(interval[0], 2))
        interval_1 = str(round(interval[1], 2))
        subtitulo = coef + ' (' + interval_0 + ' - ' + interval_1 + ')'
        subtitulo = 'p-valor = ' + str(p_valor) + ' | '+ subtitulo
        subtitulo = subtitulo + " | R² = " + str(round(results.rsquared, 2))
        
        subtitulo = str(round(Y.mean(), 2)) + " | " + subtitulo
        subtitulo = str(acre) + "=> " + subtitulo
        subtitulo = subtitulo + " | " + str(round(durbin_watson(results.resid), 2))
        
        if acre == 1:
            subtituloo = subtituloo + '\n' + subtitulo
        else:
            subtituloo = subtituloo + subtitulo
    
    plt.xlabel(subtituloo, fontproperties=fonte, fontsize=13)
    plt.show()

        ##############################################################################################

In [ ]:
tabela

In [ ]:
for i in lista2:
    print('coluna:', i)
    print('0: mean(', round(tabela[i][tabela['acreditacao'] == 0].mean(),4), ') ; median(', round(tabela[i][tabela['acreditacao'] == 0].median(),4), ')')
    print('1: mean(', round(tabela[i][tabela['acreditacao'] == 1].mean(),4), ') ; median(', round(tabela[i][tabela['acreditacao'] == 1].median(),4), ')')
    
    a = tabela[i][tabela['acreditacao'] == 0] # amostra a
    b = tabela[i][tabela['acreditacao'] == 1] # amostra b
    ttest = round(stats.ttest_ind(a,b)[1], 4)
    mann = round(stats.mannwhitneyu(a, b)[1], 4)
    print('pvalor:  '+str(ttest)+'             '+str(mann))
    
    grupos = tabela['acreditacao'].unique()

    plt.figure(figsize=(10,2))

    for grupo in grupos:

        if grupo == 0:
            label_grupo = "Sem Acreditação"
        else:
            label_grupo = "Com Acreditação"

        sns.kdeplot(tabela[tabela['acreditacao'] == grupo][i], label=label_grupo)

    plt.axvline(x = tabela[i][tabela['acreditacao'] == 0].mean(), color='red', linestyle='--', label='Média sem')
    plt.axvline(x = tabela[i][tabela['acreditacao'] == 0].median(), color='blue', linestyle='--', label='Mediana sem')

    plt.axvline(x = tabela[i][tabela['acreditacao'] == 1].mean(), color='orange', linestyle='--', label='Média com')
    plt.axvline(x = tabela[i][tabela['acreditacao'] == 1].median(), color='brown', linestyle='--', label='Mediana com')

    plt.title(i)
    plt.legend()
    plt.show()
    
    print('='*125)